In [1]:
!python -m pip install semantic-kernel

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.

In [2]:
import random
from semantic_kernel.skill_definition import sk_function

class GenerateNumberSkill:
    """
    Description: Generate a number between 3-x.
    """

    @sk_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher"
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input))) 
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

kernel = sk.Kernel()
api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Developers on an adventure. `$input` is a default variable semantic functions can use. 

In [4]:
sk_prompt = """
Write a short story about two Developers on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
"""

developer_story = kernel.create_semantic_function(prompt_template=sk_prompt,
                                              function_name="DeveloperStory",
                                              skill_name="DeveloperSkill",
                                              description="Write a short story about two Developers on an adventure",
                                              max_tokens=500,
                                              temperature=0.5,
                                              top_p=0.5)

generate_number_skill = kernel.import_skill(GenerateNumberSkill())

In [5]:
# Run the number generator
generate_number_three_or_higher = generate_number_skill["GenerateNumberThreeOrHigher"]
number_result = generate_number_three_or_higher(6)
print(number_result)

story = await developer_story.invoke_async(input=number_result.result)

print("Generating a developer story exactly {} paragraphs long: ".format(number_result.result))
print("=====================================================")
print(story)

4
Generating a developer story exactly 4 paragraphs long: 
Once upon a time, in a small town, there were two developers named Alex and Maya. They were known for their incredible coding skills and their passion for creating innovative software. One day, they received an invitation to a mysterious adventure. Curiosity sparked, they eagerly accepted the invitation and set off on their journey.

As they ventured through dense forests and crossed treacherous rivers, Alex and Maya encountered various challenges. But instead of giving up, they used their problem-solving skills to overcome each obstacle. They realized that their individual strengths complemented each other perfectly, and together, they became an unstoppable team.

Throughout their adventure, Alex and Maya met people from different backgrounds and cultures. They embraced diversity and learned from each person they encountered. They discovered that by working together and respecting one another, they could achieve great things.


## Context Variables

That works! But let's expand on our example to make it more generic. 

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the `semantic_kernel.ContextVariables` class to do hold these variables.

In [6]:
import random
from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter
from semantic_kernel import SKContext

class GenerateNumberSkillAdvanced:
    """
    Description: Generate a number between a min and a max.
    """

    @sk_function(
        description="Generate a random number between min and max",
        name="GenerateNumber"
    )
    @sk_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @sk_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value=10)
    def generate_number(self, context: SKContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"]))) 
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our DeveloperStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.

In [7]:
generate_number_skill = kernel.import_skill(GenerateNumberSkillAdvanced())
generate_number = generate_number_skill["GenerateNumber"]

In [8]:
sk_prompt = """
Write a short story about two Developers on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

developer_story = kernel.create_semantic_function(prompt_template=sk_prompt,
                                              function_name="DeveloperStory",
                                              skill_name="DeveloperSkill",
                                              description="Write a short story about two Developers on an adventure",
                                              max_tokens=500,
                                              temperature=0.5,
                                              top_p=0.5)

Now we can call this using our `invoke` function by passing in our `context_variables` in the `variables` parameter. 

In [9]:
context_variables = sk.ContextVariables(variables={
    "min": 1,
    "max": 5,
    "language": "Spanish"
})

Let's add a paragraph count to our context variables 

In [10]:
context_variables['paragraph_count'] = generate_number.invoke(variables=context_variables).result

# Pass the output to the semantic story function
story = await developer_story.invoke_async(variables=context_variables)

In [11]:
print("Generating a developer story exactly {} paragraphs long in {} language: ".format(context_variables["paragraph_count"],
                                                                                    context_variables["language"]))
print("=====================================================")
print(story)

Generating a developer story exactly 2 paragraphs long in Spanish language: 
Había una vez dos desarrolladores llamados Ana y Carlos, quienes eran los mejores amigos y siempre estaban dispuestos a enfrentar cualquier desafío juntos. Un día, recibieron una misteriosa invitación para un viaje de aventura. Emocionados, se prepararon y partieron hacia lo desconocido.

Durante su viaje, Ana y Carlos se encontraron con diversos obstáculos y problemas que requerían su habilidad y trabajo en equipo para resolverlos. A medida que avanzaban, se dieron cuenta de que la verdadera recompensa no era el tesoro que buscaban, sino la amistad y el compañerismo que habían fortalecido en su travesía. Al final, descubrieron que el verdadero tesoro era la satisfacción de superar desafíos juntos y la alegría de compartir experiencias inolvidables.

Con corazones llenos de gratitud y amistad, Ana y Carlos regresaron a casa, sabiendo que habían aprendido una valiosa lección: el verdadero valor de la vida radic